In [0]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Dense, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, Flatten
import matplotlib.pyplot as plt
from keras.datasets.mnist import load_data

In [0]:
latent_dim = 100

# DISCRIMINATOR

In [0]:
def define_discriminator(in_shape=(28,28,1)):
	model = Sequential()
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# GENERATOR

In [0]:
def define_generator(latent_dim):
	model = Sequential()
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
	return model

# GAN MODEL

In [0]:
def define_gan(g_model, d_model):
	d_model.trainable = False
	model = Sequential()
	model.add(g_model)
	model.add(d_model)
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [0]:
def load_real_samples():
	(x_train, y_train), (x_test, y_test) = load_data()
	X = expand_dims(x_train, axis=-1)
	X = X.astype('float32')
	X = X / 255.0
	return X

In [0]:
def generate_real_samples(dataset, n_samples):
	ix = randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = ones((n_samples, 1))
	return X, y

In [0]:
def generate_fake_samples(g_model, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples)
	X = g_model.predict(x_input)
	y = zeros((n_samples, 1))
	return X, y

In [0]:
def generate_latent_points(latent_dim, n_samples):
	x_input = randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [0]:
def save_plot(examples, epoch, n=10):
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i, :, :, 0], cmap='gray')
	filename = 'generated_plot_{}.png'.format(epoch+1)
	plt.savefig(filename)
	plt.close()

In [0]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	X_real, y_real = generate_real_samples(dataset, n_samples)
	loss_real, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	loss_fake, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	print('Accuracy real: {}, fake: {}' .format(acc_real*100, acc_fake*100))
	save_plot(x_fake, epoch)
	filename = 'generator_model_{}.h5'.format(epoch + 1)
	g_model.save(filename)

In [0]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
      d_loss, _ = d_model.train_on_batch(X, y)
      X_gan = generate_latent_points(latent_dim, n_batch)
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
    print('epoch: {}, iteration: {}/{}, d_loss={}, g_loss={}'.format(i+1, j+1, bat_per_epo, d_loss, g_loss))
    if (i+1) % 10 == 0:
      summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [29]:
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, d_model, gan_model, dataset, latent_dim)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 1, iteration: 234/234, d_loss=0.7042316198348999, g_loss=0.9015730023384094
epoch: 2, iteration: 234/234, d_loss=0.6630631685256958, g_loss=0.7506245970726013
epoch: 3, iteration: 234/234, d_loss=0.7120928168296814, g_loss=0.7137573957443237
epoch: 4, iteration: 234/234, d_loss=0.6842965483665466, g_loss=0.6915127038955688
epoch: 5, iteration: 234/234, d_loss=0.669113039970398, g_loss=0.734778881072998
epoch: 6, iteration: 234/234, d_loss=0.7073245048522949, g_loss=0.6882988810539246
epoch: 7, iteration: 234/234, d_loss=0.6959494948387146, g_loss=0.7014248967170715
epoch: 8, iteration: 234/234, d_loss=0.702278196811676, g_loss=0.7019523978233337
epoch: 9, iteration: 234/234, d_loss=0.6963443756103516, g_loss=0.7139756679534912
epoch: 10, iteration: 234/234, d_loss=0.6852003335952759, g_loss=0.7110025882720947
Accuracy real: 56.00000000000001, fake: 48.0
epoch: 11, iteration: 234/234, d_loss=0.690729022026062, g_loss=0.678305447101593
epoch: 12, iteration: 234/234, d_loss=0.68444